<a href="https://colab.research.google.com/github/MehraeenTimas/nlp-course/blob/main/mehraeen_persianNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install stanza requests beautifulsoup4 matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 33.0 MB/s eta 0:00:00


In [2]:
import requests, re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from collections import Counter
import matplotlib.pyplot as plt
import stanza

Initialize Persian NLP

In [3]:
stanza.download("fa")
nlp = stanza.Pipeline("fa", processors="tokenize,pos")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


web scraping

In [4]:
url = "https://trtfarsi.com"
html = requests.get(url, timeout=10).text
soup = BeautifulSoup(html, "html.parser")

titles, links = [], []
for a in soup.select("span a,div a,span, .title a,div,a")[:5]:
    title = a.get_text(strip=True)
    titles.append(title)
    links.append(a.get("href"))

texts = []
for L in links:
    try:
        if not L or not L.startswith("http"):
            L = urljoin(url, L)
        p = BeautifulSoup(requests.get(L, timeout=8).text, "html.parser")
        para = p.find("p")
        texts.append(para.get_text(strip=True)[:300] if para else "")
    except:
        texts.append("")

print(f"۵ خبر از TRT فارسی: {len(titles)} تا")

۵ خبر از TRT فارسی: 5 تا


Clean + normalize Persian text

In [5]:
raw_text = " ".join(titles + texts)
clean_text = re.sub(r'[^\u0600-\u06FF\s]', ' ', raw_text)  # keep Persian chars only
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

Process with Stanza

In [6]:
doc = nlp(clean_text)

Extract nouns and proper nouns

In [7]:
nouns = [w.text for s in doc.sentences for w in s.words if w.upos in ["NOUN", "PROPN"]]
counts = Counter(nouns)

In [9]:

persian_stopwords = {
    "ها", "های", "هایی", "ای", "آن", "این", "و", "را", "با", "در", "از", "که",
    "به", "برای", "است", "شود", "کرد", "می", "بر", "تا", "اما"
}

def clean_token(t):
    # remove suffixes like "ها" or "های" from nouns
    t = re.sub(r"(‌ها|ها|های)$", "", t)
    return t.strip()

filtered_nouns = []
for w in nouns:
    base = clean_token(w)
    if base and base not in persian_stopwords and len(base) > 2:
        filtered_nouns.append(base)

counts = Counter(filtered_nouns)


Print top 5

In [10]:
print("\n۵ اسم یا نام پرتکرار:")
for w, c in counts.most_common(5):
    print(f"   {w} → {c} بار")



۵ اسم یا نام پرتکرار:
   ترکیه → 25 بار
   ایران → 15 بار
   جمهور → 13 بار
   تحریم → 12 بار
   کشور → 11 بار


In [12]:
!jupyter nbconvert --to notebook --ClearOutputPreprocessor.enabled=True --inplace mehraeen_persianNER.ipynb


[NbConvertApp] WARNING | pattern 'mehraeen_persianNER.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--exe